In [4]:
import pandas as pd
import numpy as np
import gcsfs
import recordlinkage as rl
import matching as m
import helpers as h
%load_ext autoreload
%autoreload 2

In [5]:
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [16]:
with gcs.open('soleadify_sample_data/unified_matched_data/matches_simplified.csv') as f:
    matches_simplified = pd.read_csv(f, quotechar='"', escapechar='\\')

In order to check if there are any duplicates in this dataset, I will rely on the root_domain as the unique key

In [17]:
print('There are {:,} rows in the dataset'.format(matches_simplified.shape[0]))
print('There are {:,} unique entities in the dataset'.format(matches_simplified['root_domain_final'].nunique()))

There are 29,761 rows in the dataset
There are 26,350 unique entities in the dataset


In [18]:
matches_simplified.columns

Index(['category__facebook', 'category__google', 'category__website',
       'description__facebook', 'description__google', 'email__facebook',
       'index__facebook', 'index__google', 'index__website',
       'language__website', 'legal_name__website', 'page_type__facebook',
       'country_clean_final', 'region_clean_final', 'city_clean_final',
       'root_domain_final', 'zip_code_unified', 'phone_unified',
       'site_name_final', 'address_unified'],
      dtype='object')

In [115]:
levels_of_logic = {
    'level_1':{'country_clean_final':'value'
              ,'region_clean_final':'value'
              ,'city_clean_final':'value'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_2':{'country_clean_final':'value'
              ,'region_clean_final':'value'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_3':{'country_clean_final':'value'
              ,'region_clean_final':'isna'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_4':{'country_clean_final':'isna'
              ,'region_clean_final':'isna'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_5':{'country_clean_final':'isna'
              ,'region_clean_final':'isna'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'isna'
             }
}

In [124]:
uniques = pd.DataFrame()
deduplicated = []
for logic_dict in levels_of_logic.values():
    blocking_columns = []
    similar_columns = []
    for column in logic_dict.keys():
        if logic_dict[column] == 'value':
            blocking_columns.append(column)
        elif logic_dict[column] == 'notna':
            similar_columns.append(column)

    combinations = h.get_combinations(matches_simplified, None,blocking_columns)

    for combination in combinations:
        filtering_dict = h.get_filtering_dict(logic_dict, combination)
        filtering_query = h.get_filtering_query(filtering_dict)
        dataset = h.filter_data(matches_simplified, None, filtering_query, None)
        if dataset.shape[0]>1:
            out=m.unify_rows(dataset, blocking_columns)
            deduplicated.append(out)
        else:
            uniques = pd.concat([uniques, dataset])

In [127]:
uniques_ord = uniques.reindex(sorted(uniques.columns), axis=1)
deduplicated_df = pd.DataFrame(deduplicated)
deduplicated_df_ord = deduplicated_df.reindex(sorted(deduplicated_df.columns), axis=1)
final = pd.concat([uniques_ord, deduplicated_df_ord])

In [130]:
final['index__website'] = final['index__website'].astype('Int64')
final['index__google'] = final['index__google'].astype('Int64')
final['index__facebook'] = final['index__facebook'].astype('Int64')

In [132]:
with gcs.open('soleadify_sample_data/unified_and_unmatched_data/unified_unique.csv','w') as f:
    final.to_csv(f, index=False, quotechar='"', escapechar='\\')

In [137]:
objs = gcs.ls('soleadify_sample_data/clean_data')
for path in objs:
    if path[-4:] == '.csv':
        fileName = path.split('/')[-1]
        origin = fileName.split('_')[0]
        print(origin)
        with gcs.open(path) as f:
            if origin in ['facebook','google']:
                dataset = pd.read_csv(f, quotechar='"', escapechar='\\')
            else:
                dataset = pd.read_csv(f)
        umatched = dataset.loc[~dataset.index.isin(final['index__'+origin])]
        with gcs.open('soleadify_sample_data/unified_and_unmatched_data/'+origin+'_unmatched.csv','w') as f:
            umatched.to_csv(f, index=False, quotechar='"', escapechar='\\')

facebook
google
website
